# My tentative "unit test" of the Models class...

- Implement "BM" investment (select largest 3 assets, using market cap weights)
- Implement "size" (select smallest 5 assets, using stddev as weights).
- Compare with Excel solution

# Required packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Just to show pretty figures:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import os

# Module with the class that solves the problem:
from Models import *

# Data  (and set the tolerance for equality of results)

In [2]:
tolerance = 1e-8

data_dir = "../data/"

excel_file = 'testData.xlsx'

data = {}

for x in ['Prices', 'BM', 'Size', 'StdDev', 'Output 5Size-StdDev', 'Output 3BM-MktCap', 'Returns']:
    df = pd.read_excel(os.path.join(os.getcwd(), data_dir, excel_file), sheet_name=x)
    
    # Fix the date:
    df.date = pd.to_datetime(df.date, format='%Y-%m-%d')
    df = df.set_index('date')
    data[x] = df.copy()

# Funtion to test equal results

In [3]:
def compare2Excel(mod, solution):
    
    print('Model name: ', mod.name)
    
    # returns in each period:
    mod.calculatePerformance()
    
    # check if within tolerance from excel result:
    if abs(mod.performance[mod.name] - data[solution]['Portfolio']).max() < tolerance:
        print('\n OK 1 period returns for tolerance ', tolerance)
    else:
        print('\n FAIL 1 period returns for tolerance ', tolerance)
        
    # Cumulative returns:
    mod.compareCumulative(compareStarts=False, rLabel=False, showGraph=False)

    if abs(mod.compare['Cum. ' + mod.name] - data[solution]['Cumulative']).max() < tolerance:
        print('\n OK cumulative returns for tolerance ', tolerance)
    else:
        print('\n FAIL cumulative returns for tolerance ', tolerance)
        
    print('\n')


# Put models in a dictionary

In [4]:
models_dict = {}
solutionExcel = {}

# Model 1
#########

key = ('BM')

models_dict[key] = Models(key, 
                          name='BM',
                          assetsN=3,
                          targetLow=False)
# The 3 matrices:
models_dict[key].classifVariable = data['BM']
models_dict[key].baseWeights = data['Size']
models_dict[key].basePrices = data['Prices']
#models_dict[key].baseReturns = data['Returns']

solutionExcel[key] = 'Output 3BM-MktCap'


# Model 2
#########

key = ('MktCap')

models_dict[key] = Models(key, 
                          name='Size',
                          assetsN=5,
                          targetLow=True)

# The 3 matrices:
models_dict[key].classifVariable = data['Size']
models_dict[key].baseWeights = data['StdDev']
models_dict[key].basePrices = data['Prices']
#models_dict[key].baseReturns = data['Returns']

solutionExcel[key] = 'Output 5Size-StdDev'

# Test all models

In [5]:
for key in models_dict:
    compare2Excel(models_dict[key], solutionExcel[key])

Model name:  BM

 OK 1 period returns for tolerance  1e-08

 OK cumulative returns for tolerance  1e-08


Model name:  Size

 OK 1 period returns for tolerance  1e-08

 OK cumulative returns for tolerance  1e-08


